In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *

import os


In [2]:
data_dir = f'{main_dir}/Snapshots/1/05-LabelEncoded-last'
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/Catboost')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'label_encoded_data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'bad_for_training_features',
           params={
               'input_directory': data_dir,
               'file': 'bad_for_training_features.yaml'
           })
p.save()
p.run(verbose=True)

bad_features = p.data['bad_for_training_features']

if 'isFraud' in bad_features:
    bad_features.remove('isFraud')

p.save()


---------------------------
0: LoaderNode [2019-09-29 19:10:07]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'label_encoded_data.pkl'}
---------------------------
1: LoaderNode [2019-09-29 19:10:16]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'numerical_columns.yaml'}
---------------------------
2: LoaderNode [2019-09-29 19:10:16]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'categorical_columns.yaml'}
---------------------------
3: LoaderNode [2019-09-29 19:10:16]
params:
 {'input_directory': '../../../Snapshots/1/05-LabelEncoded-last', 'file': 'bad_for_training_features.yaml'}


## Code to make pipeline node

In [4]:
# df = p.data['data']
train = p.data['data'][p.data['data'].isFraud >= 0]
test =  p.data['data'][p.data['data'].isFraud < 0]


X = train.drop('isFraud',axis=1)
y = train['isFraud']

test =  test.drop('isFraud',axis=1)

X.replace(np.inf, np.nan, inplace=True)
X.replace(-np.inf, np.nan, inplace=True)
test.replace(np.inf, np.nan, inplace=True)
test.replace(-np.inf, np.nan, inplace=True)

# categorical_features = [c for c, col in enumerate(X.columns) if col in p.data['categorical_columns']]

p.data = None
train_idx = train.index
del train
gc.collect()

for col in X.columns:
    if str(X[col].dtype) == 'datetime64[ns]':
        print (col, X[col].dtype)
        X[col] = pd.to_timedelta(X[col]).map(lambda qq:qq.days)
        test[col] = pd.to_timedelta(test[col]).map(lambda qq:qq.days)
#         X[col] = X[col].astype(datetime.datetime)

for col in X.columns:
    if X[col].dtype == np.float64:
        print (col)
        X[col] = X[col].astype(np.float32)
        test[col] = test[col].astype(np.float32)
#         test[col] = pd.to_timedelta(test[col]).map(lambda qq:qq.days)
#         X[col] = X[col].astype(datetime.datetime)

X.to_pickle(r'e:\Kaggle\temp\X.pkl')
test.to_pickle(r'e:\Kaggle\temp\test.pkl')
y.to_pickle(r'e:\Kaggle\temp\y.pkl')

In [2]:
X = pd.read_pickle(r'e:\Kaggle\temp\X.pkl')
test= pd.read_pickle(r'e:\Kaggle\temp\test.pkl')
y= pd.read_pickle(r'e:\Kaggle\temp\y.pkl')

In [3]:
from sklearn.model_selection import GroupKFold

NFOLDS = 6
folds = GroupKFold(n_splits=NFOLDS)


In [4]:
import yaml
categorical_features = yaml.load(open(r'd:\Documents\Private\Kaggle\Baydin\Snapshots\1\05-LabelEncoded-last\categorical_columns.yaml','r'),Loader=yaml.FullLoader)
numerical_features = yaml.load(open(r'd:\Documents\Private\Kaggle\Baydin\Snapshots\1\05-LabelEncoded-last\numerical_columns.yaml','r'),Loader=yaml.FullLoader)
bad_features = yaml.load(open(r'd:\Documents\Private\Kaggle\Baydin\Snapshots\1\05-LabelEncoded-last\bad_for_training_features.yaml','r'),Loader=yaml.FullLoader)

categorical_features = [f for f in categorical_features if f not in bad_features]
all_features =  [f for f in X.columns if f not in bad_features]

In [5]:
from BayDS.lib.training import *

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [ ]:
# nans_groups = {}
# temp_df = train.isna()
# temp_df2 = test.isna()

# nans_df = pd.concat([temp_df, temp_df2])

# for col in list(nans_df):
#     cur_group = nans_df[col].sum()
#     if cur_group>0:
#         try:
#             nans_groups[cur_group].append(col)
#         except:
#             nans_groups[cur_group]=[col]

# add_category = []
# for col in nans_groups:
#     if len(nans_groups[col])>1:
#         train['nan_group_'+str(col)] = np.where(temp_df[nans_groups[col]].sum(axis=1)>0,1,0).astype(np.int8)
#         test['nan_group_'+str(col)]  = np.where(temp_df2[nans_groups[col]].sum(axis=1)>0,1,0).astype(np.int8)
#         add_category.append('nan_group_'+str(col))

In [24]:
# total_items = len(X)
# keep_cols = ['isFraud','C3_fq_enc']
# for col in numerical_features:
#         cur_dominator = list(X[col].fillna(-999).value_counts())[0]
#         if (cur_dominator/total_items > 0.85) and (col not in keep_cols):
#             cur_dominator = X[col].fillna(-999).value_counts().index[0]
#             print('Column:', col, ' | Dominator:', cur_dominator)
#             X[col] = np.where(X[col].fillna(-999)==cur_dominator,1,0)
#             test[col] = np.where(test[col].fillna(-999)==cur_dominator,1,0)

#             X[col] = X[col].fillna(-999).astype(int)
#             test[col] = test[col].fillna(-999).astype(int)

#             if col not in categorical_features:
#                 categorical_features.append(col)

Column: BrowserAge  | Dominator: -999.0
Column: card3_global_count  | Dominator: 956845
Column: D6_global_count  | Dominator: 899261
Column: D7_global_count  | Dominator: 998181
Column: D8_global_count  | Dominator: 947967
Column: D9_global_count  | Dominator: 947967
Column: D12_global_count  | Dominator: 963260
Column: D13_global_count  | Dominator: 911895
Column: D14_global_count  | Dominator: 919850
Column: C3_global_count  | Dominator: 1081890
Column: C7_global_count  | Dominator: 961237
Column: device_version_global_count  | Dominator: -999.0
Column: id_30_global_count  | Dominator: 949007
Column: id_33_global_count  | Dominator: 953271
Column: OS_global_count  | Dominator: 949007
Column: OSVersion_global_count  | Dominator: 949007
Column: addr2_global_count  | Dominator: 956415
Column: dist2_global_count  | Dominator: 1023168
Column: id_14_global_count  | Dominator: 945830
Column: id_18_global_count  | Dominator: 1001243
Column: id_21_global_count  | Dominator: 1087013
Column: id

KeyboardInterrupt: 

In [6]:
params = {
#                 'n_estimators':5000,
                'learning_rate': 0.07,
                'eval_metric':'AUC',
                'loss_function':'Logloss',
                'random_seed':42,
                'metric_period':500,
                'od_wait':500,
                'task_type':'GPU',
                'depth': 8,
                #'colsample_bylevel':0.7,
                } 
n_fold = 6
folds = GroupKFold(n_splits=n_fold)
train_options = {
    "model_type":'cat',
    "params": params,
    'early_stopping_rounds': 1000,
    'n_estimators': 5000,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
    
}

In [7]:
result_dict_cb_new_id = train_model_classification_vb(X=X, X_test=test, columns=all_features,
                                                   y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             categorial_columns = categorical_features,
                                             plot_feature_importance=True,
                                             verbose=100, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             n_jobs=-1, groups=X['DT_M'])


Fold 1 started at Sun Sep 29 19:47:16 2019
0:	learn: 0.8914227	test: 0.7816849	best: 0.7816849 (0)	total: 2.75s	remaining: 3h 48m 52s
500:	learn: 0.9787215	test: 0.9237576	best: 0.9238504 (493)	total: 18m 4s	remaining: 2h 42m 21s
1000:	learn: 0.9831299	test: 0.9302061	best: 0.9302672 (992)	total: 36m 7s	remaining: 2h 24m 17s
1500:	learn: 0.9863489	test: 0.9328698	best: 0.9329494 (1479)	total: 54m 5s	remaining: 2h 6m 5s
2000:	learn: 0.9892587	test: 0.9342164	best: 0.9342164 (2000)	total: 1h 12m 18s	remaining: 1h 48m 23s
2500:	learn: 0.9915988	test: 0.9343024	best: 0.9344447 (2491)	total: 1h 30m 36s	remaining: 1h 30m 31s
3000:	learn: 0.9935500	test: 0.9352641	best: 0.9354221 (2893)	total: 1h 48m 50s	remaining: 1h 12m 29s
bestTest = 0.9354220629
bestIteration = 2893
Shrink model to first 2894 iterations.


CatBoostError: bad allocation

In [11]:
pd.DataFrame(result_dict_lgb['oof'], columns=['isFraud'], index=train_idx).to_csv(f'{main_dir}/Snapshots/1/Training2509/oof_all_data_best_lgb.csv')

pd.DataFrame(result_dict_lgb['prediction'], columns=['isFraud'], index=test.index).to_csv(f'{main_dir}/Snapshots/1/Training2509/prediction_all_data_best_lgb.csv')

sample_submission = pd.read_csv(data_path + '/sample_submission.csv').set_index('TransactionID')

sub1 = pd.DataFrame(result_dict_lgb['prediction'], columns=['isFraud'], index=test.index)

sample_submission['isFraud'] = sub1#*0.5 + sub2*0.25 + sub3*0.25

sample_submission.to_csv(f'{main_dir}/lgb_sub.csv')